
... ***CURRENTLY UNDER DEVELOPMENT*** ...


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Reconstruction, RBF_Validation



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')

# input files: dataset, subset (MDA classification), and target (SWAN simulations output)

# sea 
wvs_sea_dataset = db.Load_NEARSHORE_FULL_sea()
wvs_sea_subset = db.Load_NEARSHORE_MDA_sea()
wvs_sea_target = db.Load_NEARSHORE_TARGET_sea()

# swells 
wvs_swell_dataset = db.Load_NEARSHORE_FULL_swell()
wvs_swell_subset = db.Load_NEARSHORE_MDA_swell()
wvs_swell_target = db.Load_NEARSHORE_TARGET_swell()


# remove nans (if any) from data
def fix_target_nans(subset, target):
    'remove NaN data indexes from subset and target. RBF does not handle NaNs'
    
    r_nan = target.isnull().any(axis=1)  # find any row with nans

    if r_nan.any():
        # log
        print('remove nan data found at target:')
        print(target[r_nan])
        
        # clean data
        target = target[~r_nan]
        subset = subset[~r_nan]
        
    return subset, target

wvs_sea_subset, wvs_sea_target = fix_target_nans(wvs_sea_subset, wvs_sea_target)
wvs_swell_subset, wvs_swell_target = fix_target_nans(wvs_swell_subset, wvs_swell_target)


# ensure dataset and subset have same variables and column order
vns_ds = ['hs', 'tp', 'dir']
wvs_sea_dataset = wvs_sea_dataset[vns_ds].values
wvs_sea_subset = wvs_sea_subset[vns_ds].values
wvs_swell_dataset = wvs_swell_dataset[vns_ds].values
wvs_swell_subset = wvs_swell_subset[vns_ds].values


# select target variables
vns_tgt = ['Hsig', 'TPsmoo', 'Dir']
wvs_sea_target = wvs_sea_target[vns_tgt].values
wvs_swell_target = wvs_swell_target[vns_tgt].values



remove nan data found at target:
         lat       lon  Hsig  Tm02  Dir  Dspr  TPsmoo
case                                                 
49    9.4165  167.4679   0.0   NaN  NaN   NaN     NaN


In [3]:
# RBF settings

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]      # scalar (hs, tp)
ix_directional_subset = [2]   # directional (dir)

# target - scalar / directional indexes
ix_scalar_target = [0,1]      # scalar (Hsig, Tpsmoo, Dir)
ix_directional_target = [2]   # directional (Dir)



## RBF Reconstruct SEA Waves

In [4]:
# --------------------------------------
# RBF Interpolation

sea_recon = RBF_Reconstruction(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target,
    wvs_sea_dataset)

# store it as pandas dataframe
df_sea_recon = pd.DataFrame(data=sea_recon, columns=vns_tgt)
print(df_sea_recon)

# store data
db.Save_NEARSHORE_RECONSTRUCTION_sea(df_sea_recon)


# TODO: add scatter plot for RBF validation


ix_scalar: 0,  optimization: 48.37 | interpolation: 51.28
ix_scalar: 1,  optimization: 86.70 | interpolation: 50.92
ix_directional: 2,  optimization: 102.84 | interpolation: 113.55
            Hsig    TPsmoo         Dir
0       0.208115  2.429508  101.707861
1       0.265490  2.606427   92.338007
2       0.290903  2.989002  114.112774
3       0.573906  5.413579  112.642724
4       0.347916  3.074125  106.282561
...          ...       ...         ...
389576  0.288512  2.783956  104.232569
389577  0.216576  2.300218   83.985624
389578  0.349989  2.931648   93.273763
389579  0.669135  9.223054  108.896141
389580  0.271511  2.646362   96.297690

[389581 rows x 3 columns]


In [5]:
# --------------------------------------
# RBF Validation

valid_sea = RBF_Validation(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target)

print(valid_sea)
 

RBFs Kfold Validation: 1/3
ix_scalar: 0,  optimization: 20.67 | interpolation: 0.02
ix_scalar: 1,  optimization: 40.40 | interpolation: 0.02
ix_directional: 2,  optimization: 42.66 | interpolation: 0.05
mean squared error : 9.102885330847544

RBFs Kfold Validation: 2/3
ix_scalar: 0,  optimization: 16.62 | interpolation: 0.02
ix_scalar: 1,  optimization: 43.46 | interpolation: 0.02
ix_directional: 2,  optimization: 38.90 | interpolation: 0.04
mean squared error : 257.50994829365555

RBFs Kfold Validation: 3/3
ix_scalar: 0,  optimization: 17.58 | interpolation: 0.02
ix_scalar: 1,  optimization: 39.51 | interpolation: 0.02
ix_directional: 2,  optimization: 37.69 | interpolation: 0.04
mean squared error : 257.22081626526744

<xarray.Dataset>
Dimensions:             (n_split: 3, test: 167, train: 334)
Coordinates:
  * n_split             (n_split) int64 0 1 2
Dimensions without coordinates: test, train
Data variables:
    mean_squared_error  (n_split) float64 9.103 257.5 257.2
    train_ind


## RBF Reconstruct SWELL Waves

In [6]:
# --------------------------------------
# RBF Interpolation

swl_recon = RBF_Reconstruction(
    wvs_swell_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swell_target, ix_scalar_target, ix_directional_target,
    wvs_swell_dataset)

# store it as pandas dataframe
df_swl_recon = pd.DataFrame(data=swl_recon, columns=vns_tgt)
print(df_swl_recon)

# store data
db.Save_NEARSHORE_RECONSTRUCTION_swell(df_swl_recon)


# TODO: add scatter plot for RBF validation


ix_scalar: 0,  optimization: 35.78 | interpolation: 266.48
ix_scalar: 1,  optimization: 80.31 | interpolation: 276.14
ix_directional: 2,  optimization: 78.87 | interpolation: 541.86
             Hsig     TPsmoo         Dir
0        0.979251   9.754465   13.341103
1        0.987271  12.053734  353.093848
2        0.995833  15.582940  355.326341
3        1.002170  10.874706  331.869214
4        0.998977  13.655417  347.461648
...           ...        ...         ...
2173228  0.993051   8.884842   65.933639
2173229  0.995582   7.834524   75.671123
2173230  1.003759   9.711813   58.693232
2173231  1.002928   6.917088   76.531611
2173232  1.012554   7.262348   63.244294

[2173233 rows x 3 columns]


In [7]:
# --------------------------------------
# RBF Validation

valid_swl = RBF_Validation(
    wvs_swell_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swell_target, ix_scalar_target, ix_directional_target)

print(valid_swl)


RBFs Kfold Validation: 1/3
ix_scalar: 0,  optimization: 16.92 | interpolation: 0.02
ix_scalar: 1,  optimization: 36.10 | interpolation: 0.02
ix_directional: 2,  optimization: 30.91 | interpolation: 0.04
mean squared error : 1.2230857213705364

RBFs Kfold Validation: 2/3
ix_scalar: 0,  optimization: 17.23 | interpolation: 0.02
ix_scalar: 1,  optimization: 36.74 | interpolation: 0.02
ix_directional: 2,  optimization: 34.11 | interpolation: 0.04
mean squared error : 1.225287495847731

RBFs Kfold Validation: 3/3
ix_scalar: 0,  optimization: 15.68 | interpolation: 0.02
ix_scalar: 1,  optimization: 34.48 | interpolation: 0.02
ix_directional: 2,  optimization: 34.11 | interpolation: 0.04
mean squared error : 0.4157997612855431

<xarray.Dataset>
Dimensions:             (n_split: 3, test: 167, train: 333)
Coordinates:
  * n_split             (n_split) int64 0 1 2
Dimensions without coordinates: test, train
Data variables:
    mean_squared_error  (n_split) float64 1.223 1.225 0.4158
    train_in

## Methodology Validation

In [8]:
# TODO: validate with buoy?
